In [1]:
"""
Create subword dictionary to breakdown words for embedding training
"""

import pickle
from collections import Counter


In [2]:
#Load Assets

with open(r'D:\dstore\nlp\w2v\wcounts16', 'rt', encoding='utf8') as f:
    wlst = [x.strip().split() for x in f.readlines()]

#Filter out words w/ < 256 occurences 
wlst = Counter({x[1]: int(x[0]) for x in wlst if int(x[0]) > 128})

#Count letters in all words
lcount = Counter()
for x in wlst:
    for y in x:
        lcount[y] += 1

#Filter out letters w/ < 256 occurences
lcount = {x[0]: x[1] for x in lcount.items() if x[1] >= 256}
alpha = [*lcount, '_', "'", '[UNK]']
wlst = {' '.join(list(x[0])): x[1] for x in wlst.items()}

#Remove all words with unknown letters
temp = []
for x in wlst:
    for y in x.split():
        if y not in alpha:
            temp.append(x)
            break
for x in temp:
    wlst.pop(x)

#Remove all words that do not have a vowel
vowels = ['a', 'e', 'i', 'o', 'u', 'y', 'á', 'é', 'ö']

wlst = Counter({f'_{x[0]}_': x[1] for x in wlst.items() if len(x[0]) > 1 and any({y for y in vowels if y in x[0]}) and len(x[0]) > 6})
stubs = {f'{x[:-3]}_' for x in wlst}


In [17]:
if '_e d u c a t_' in stubs:
    print('yes')

yes


In [3]:
#Precombine common suffix subwords first to precondition byte encoding
primes = set()

with open(r'D:\dstore\nlp\w2v\prefs2', 'rt', encoding='utf8') as f:
    for x in f.readlines():
        primes.add(x.strip())

with open(r'D:\dstore\nlp\w2v\suffs2', 'rt', encoding='utf8') as f:
    for x in f.readlines():
        primes.add(x.strip())

isubwords = [('s_', 'es_'), ('r_', ('er_', 'or_')), ('d_', ('ed_'))]

primes = sorted(primes, key=lambda x: len(x), reverse=True)
mods = []
for word in wlst:
    temp = []
    for subw in primes:
        esubw = ' '.join(list(subw)).replace(' _', '_').replace('_ ', '_')
        if esubw[0] == '_':
            rep = word.replace(f'{esubw} ', '_')
            if rep != word and rep in wlst:
                temp.append(esubw)
        elif esubw[-1] == '_':
            rep = word.replace(f' {esubw}', '_')
            if rep != word and (rep in wlst or rep in stubs):
                temp.append(esubw)
    for subw in isubwords:
        
    if len(temp) > 0:
        for x in temp[::-1]:
            for y in temp:
                if x != y and x in y:
                    temp.remove(x)
                    break
        mods.append((word, temp))




""" subwords = set()

with open(r'D:\dstore\nlp\w2v\prefs', 'rt', encoding='utf8') as f:
    for x in f.readlines():
        subwords.add(x.strip())

with open(r'D:\dstore\nlp\w2v\suffs', 'rt', encoding='utf8') as f:
    for x in f.readlines():
        subwords.add(x.strip())

for subw in sorted(subwords, key=lambda x: len(x), reverse=True):
    esubw = ' '.join(list(subw)).replace(' _', '_').replace('_ ', '_')
    temp = []
    for word in wlst:
        if esubw in word:
            if esubw[0] == '_' and word.replace(f'{esubw} ', '_') in wlst:
                temp.append((word, word.replace(esubw, subw)))
            elif esubw[-1] == '_' and word.replace(f' {esubw}', '_') in wlst:
                temp.append((word, word.replace(esubw, subw)))
    for x in temp:
        if len(x) > 1:
            print(x)
        wlst[x[1]] += wlst[x[0]]
        wlst.pop(x[0]) """

""" subwords = subwords.union(primes)
#Convert all 2/3 letter words w/ > 4000 occurences into subwords directly
temp = []
for x in wlst.most_common():
    if len(x[0]) in [4, 6]:
        temp.append(x)

for x in temp[:975]:
    subwords.add(''.join(x[0].split()))
    wlst.pop(x[0])

subwords = subwords.union(set(['_i_', "_i'm_", '_a_', '_when_', '_been_', '_have_', '_two_', '_they_', '_who_', '_not_', '_new_', '_but_', '_her_', '_she_', '_after_', '_one_', '_their_', '_or_', '_has_', '_first_', '_had_', '_be_', '_also_', '_this_', '_are_', '_which_', '_were_', '_an_', '_it_', '_his_', '_from_', '_at_', '_that_', '_he_', '_by_', '_with_', '_as_', '_for_', '_on_', '_is_', '_was_', '_to_', '_and_', '_in_', '_of_', '_the_']))

#Remove fully merged words from word list
temp = []
for x in wlst:
    if ' ' not in x:
        temp.append(x)
for x in temp:
    subwords.add(wlst.pop(x))

del temp, lcount, vowels """




' subwords = subwords.union(primes)\n#Convert all 2/3 letter words w/ > 4000 occurences into subwords directly\ntemp = []\nfor x in wlst.most_common():\n    if len(x[0]) in [4, 6]:\n        temp.append(x)\n\nfor x in temp[:975]:\n    subwords.add(\'\'.join(x[0].split()))\n    wlst.pop(x[0])\n\nsubwords = subwords.union(set([\'_i_\', "_i\'m_", \'_a_\', \'_when_\', \'_been_\', \'_have_\', \'_two_\', \'_they_\', \'_who_\', \'_not_\', \'_new_\', \'_but_\', \'_her_\', \'_she_\', \'_after_\', \'_one_\', \'_their_\', \'_or_\', \'_has_\', \'_first_\', \'_had_\', \'_be_\', \'_also_\', \'_this_\', \'_are_\', \'_which_\', \'_were_\', \'_an_\', \'_it_\', \'_his_\', \'_from_\', \'_at_\', \'_that_\', \'_he_\', \'_by_\', \'_with_\', \'_as_\', \'_for_\', \'_on_\', \'_is_\', \'_was_\', \'_to_\', \'_and_\', \'_in_\', \'_of_\', \'_the_\']))\n\n#Remove fully merged words from word list\ntemp = []\nfor x in wlst:\n    if \' \' not in x:\n        temp.append(x)\nfor x in temp:\n    subwords.add(wlst.pop(x))

In [4]:
for x in mods:
    new_word = x[0]
    for y in x[1]:
        merged = ''.join(y.split())
        new_word = new_word.replace(y, merged)
    wlst[new_word] += wlst[x[0]]
    wlst.pop(x[0])

In [ ]:
[x for x in mods if len(x[1]) > 1]

In [8]:
wlst.most_common()

[('_o t h e r_', 3090563),
 ('_d u r ing_', 3080711),
 ('_t i m e_', 3032872),
 ('_i n t o_', 2817869),
 ('_t h e r e_', 2632863),
 ('_m o r e_', 2484511),
 ('_o v e r_', 2420960),
 ('_s c h o o l_', 2413586),
 ('_w o u l d_', 2395667),
 ('_y e a r_', 2342691),
 ('_y e a r s_', 2315301),
 ('_o n l y_', 2269424),
 ('_be t w e e n_', 2250201),
 ('_l a t e r_', 2246834),
 ('_w h e r e_', 2228181),
 ('_m o s t_', 2178584),
 ('_t h r e e_', 2172672),
 ('_t h e n_', 2087621),
 ('_s o m e_', 2077223),
 ('_w h i l e_', 2053584),
 ('_a b o u t_', 2043515),
 ('_w o r l d_', 2020214),
 ('_c i t y_', 2004404),
 ('_s u c h_', 2000311),
 ('_s t a t e_', 1999408),
 ('_m a d e_', 1996537),
 ('_u n i v e r s ity_', 1953905),
 ('_u n d e r_', 1943360),
 ('_n a t i o n al_', 1939573),
 ('_k n o w n_', 1937752),
 ('_u s e d_', 1930084),
 ('_p a r t_', 1911701),
 ('_be c a m e_', 1889453),
 ('_f i l m_', 1854465),
 ('_u n i t e d_', 1839511),
 ('_m a n y_', 1831959),
 ('_s e c o n d_', 1821689),
 ('_t h a 

In [8]:
#Counts pairs of subwords
candidate_frags = Counter()
for word in wlst:
    word = word.split()
    for i, _ in enumerate(word[:-1]):
        candidate_frags[f'{word[i]} {word[i+1]}'] += wlst[' '.join(word)]

candidate_frags = Counter({x[0]: x[1] for x in candidate_frags.items()})

In [11]:
#Byte encoding algorithm
#Processes the 16 most common pairs, counts pairs repeats
#Check if subword occurs for any word in word_list.
#If subword pair is found, merge pair and replace the pair with the combined subword

while len(subw) < 65536:
    count = 0
    while count < 16:
        sub = candidate_frags.most_common()[0][0]
        matches = []
        csub = sub.split()
        for word in wlst:
            if sub not in word:
                continue
            if f'{sub} ' != word[:len(sub) + 1] and f' {sub}' != word[len(word) - len(sub) - 1:] and f' {sub} ' not in word and len(sub) != len(word):
                continue
            word = word.split()
            is_match = False
            idx = []
            outp = word.copy()
            for i, y in enumerate(word[:-1]):
                if is_match:
                    is_match = False
                    continue
                if csub[0] == y and csub[1] == word[i+1]:
                    idx.append(i)
                    is_match = True
            for i, pos in enumerate(idx):
                outp.insert(pos + i, ''.join(csub))
            for i, pos in enumerate(idx):
                outp.pop(pos + 1 - i)
                outp.pop(pos + 1 - i)
            matches.append((' '.join(word), ' '.join(outp)))
        for m in matches:
            wlst[m[1]] += wlst[m[0]]
            wlst.pop(m[0])
        candidate_frags.pop(sub)
        subw.add(sub.replace(' ', ''))
        count += 1

    #Remove fully combined words from list
    for x in list(wlst):
        if ' ' not in x:
            wlst.pop(x)

    #Count pairs
    candidate_frags = Counter()
    for word in wlst:
        word = word.split()
        for i, _ in enumerate(word[:-1]):
            candidate_frags[f'{word[i]} {word[i+1]}'] += wlst[' '.join(word)]

    candidate_frags = Counter({x[0]: x[1] for x in candidate_frags.items()})

    """ for x in candidate_frags.most_common()[:10]: print(x)
    print(len(candidate_frags), '\n')
    for x in wlst.most_common()[:10]: print(x)
    print(len(wlst), '\n') """


In [13]:
subw

{'hmandu_',
 '_releasing_',
 '_reverber',
 '_pollard_',
 '_darmstadt_',
 'care_',
 '_haplo',
 'hay_',
 '_haircut_',
 '_epidemiologist_',
 '_volatile_',
 'erville_',
 '_directorate_',
 '_toby_',
 '_abruptly_',
 '_feared_',
 '_orchest',
 '_germ',
 '_jermaine_',
 '_weaker_',
 '_symmetry_',
 '_oddly_',
 '_smock_',
 'silanti_',
 'eboygan_',
 'proofing_',
 'éb',
 '_ghazal_',
 'étaro_',
 '_valley_',
 '_ting',
 '_felon_',
 '_onward_',
 'ounslow_',
 '_winemaker_',
 '_welter',
 '_aor',
 '_ridgewood_',
 '_dé',
 '_mercenary_',
 '_girard_',
 '_rod',
 '_accessories_',
 '_insiders_',
 'ugu_',
 '_prefecture_',
 '_mayfield_',
 '_suffer',
 '_undi',
 'agic_',
 '_wishart_',
 '_leper_',
 '_eyck_',
 '_maggie_',
 'ircea_',
 'ridors_',
 '_heraklion_',
 '_scholarship_',
 '_doha_',
 '_keto',
 '_jamestown_',
 '_mormonism_',
 '_forgo_',
 '_jus',
 '_datu_',
 'not_',
 '_vadod',
 '_bluet',
 'amore_',
 '_sharap',
 '_resist_',
 '_stephan_',
 '_beit_',
 '_therapies_',
 '_criti',
 '_bibliographic_',
 '_superflu',
 '_ref

In [14]:
for x in alpha:
    subw.add(x)

version = 'x3'

with open(f'D:/dstore/nlp/w2v/wlst-{version}', 'wb') as f:
    pickle.dump(wlst, f)

with open(f'D:/dstore/nlp/w2v/subw-{version}', 'wb') as f:
    pickle.dump(subw, f)


In [4]:
version = 'x3'
with open(f'D:/dstore/nlp/w2v/wlst-{version}', 'rb') as f:
    wlst = pickle.load(f)

with open(f'D:/dstore/nlp/w2v/subw-{version}', 'rb') as f:
    subw = pickle.load(f)

In [37]:
subw = sorted(subw, key=lambda x: len(x), reverse=True)

In [26]:

def fragment(word):
    tracker = word
    for x in subw:
        if x in tracker:
            tracker = tracker.replace(x, ' ').strip()
            word = word.replace(x, f' {x} ').strip()
    for x in list(tracker):
        if x not in subw:
            word = word.replace(x, ' [UNK] ').strip()
    while '  ' in word:
        word = word.replace('  ', ' ')
    word = word.replace('_ ', '_').replace(' _', '_')
    return word


In [18]:
with open(r'D:\dstore\nlp\w2v\wcounts16', 'rt', encoding='utf8') as f:
    twords = [f'_{x.strip().split()[1]}_' for x in f.readlines()]

In [31]:
if '_surround_' in subw:
    print('yes')

yes


In [38]:
fragment('_surrounds_')

'_surr ound s_'

In [ ]:
for x in twords[::-1][13000:14000]:
    print(fragment(x))